In [1]:
import numpy as np
from numpy.linalg import inv, eig
from scipy.linalg import fractional_matrix_power as power

import pandas as pd

import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
from sklearn.decomposition import PCA
from sklearn.cross_decomposition import PLSRegression

from collections import namedtuple

/anaconda3/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
from rraff import rraff
from olsnw import olsnw

In [3]:
t0 = pd.to_datetime('197401',format='%Y%m') 
t1 = pd.to_datetime('201612',format='%Y%m') 

In [4]:
z = pd.read_excel('Portfolio_ret.xlsx',sheet_name='FF5')
z.index = pd.to_datetime(z.index,format="%Y%m")
# riskfree rate
rf = z['RF'][t0:t1].values 
rf = rf.reshape(-1,1)
#excess mkt return
mkt = z['Mkt-RF'][t0:t1].values 
mkt = mkt.reshape(-1,1)

In [6]:
z = pd.read_excel('Portfolio_ret.xlsx',sheet_name='Anomaly',header=1)
z.index = pd.to_datetime(z.index,format="%Y%m")
z = z.loc[t0:t1,:]

f = sc.fit_transform(z.iloc[:,:10]) # benchmark: 1, 3, 5, 6,10 factors
G = z.values

T, L = G.shape

# Factor Extraction
Run the corresponding section for certain set of testing assets

In [42]:
### Factors: 48 Industry Portfolios ###
z = pd.read_excel('Portfolio_ret.xlsx',sheet_name='FF48vw',header=1)
z.index = pd.to_datetime(z.index,format="%Y%m")
z = z.loc[t0:t1,:]

N = z.shape[1] # number of portfolios

R48 = z - rf@np.ones((1,N))
R48 = R48.values

rraf, pcaf, plsf = rraff(R48,G,10)
rraf = sc.fit_transform(rraf)
pcaf = sc.fit_transform(pcaf)
plsf = sc.fit_transform(plsf)

R = R48/100

In [38]:
###Factors: 202 Dacheng's target ###
z = pd.read_excel('Portfolio_ret.xlsx',sheet_name='Dacheng202vw',header=1)
z.index = pd.to_datetime(z.index,format="%Y%m")
z = z.loc[t0:t1,:]

N = z.shape[1] # number of portfolios

R202 = z - rf@np.ones((1,N)) 
R202 = R202.values

rraf, pcaf, plsf = rraff(R202,G,10)
rraf = sc.fit_transform(rraf)
pcaf = sc.fit_transform(pcaf)
plsf = sc.fit_transform(plsf)

R = R202/100

# Pricing performance

In [43]:
### Always run this section for restults ###
NF = [1,3,5,6,10]
res_f = np.zeros((len(NF),N,3))
res_rra = np.zeros((len(NF),N,3))
res_pca = np.zeros((len(NF),N,3))
res_pls = np.zeros((len(NF),N,3))

for i in range(N):
    for j in range(len(NF)):
        Ri = R[:,i]
        Ttemp = len(Ri)
        
        b,tstat,pvals,bse,adjR2,yhat = olsnw(Y=Ri,X=f[:,:NF[j]])
        adj = (Ttemp-1)/(Ttemp-1-NF[j])
        temp1 = np.sum((Ri-yhat)**2)*adj
        temp2 = np.sum(Ri**2);
        temp3 = np.sum((Ri-yhat+b[0])**2)*adj/Ttemp
        res_f[j,i,:] = [temp1, temp2, temp3]
        
        b,tstat,pvals,bse,adjR2,yhat = olsnw(Y=Ri,X=rraf[:,:NF[j]])
        adj = (Ttemp-1)/(Ttemp-1-NF[j])
        temp1 = np.sum((Ri-yhat)**2)*adj
        temp2 = np.sum(Ri**2);
        temp3 = np.sum((Ri-yhat+b[0])**2)*adj/Ttemp
        res_rra[j,i,:] = [temp1, temp2, temp3]
        
        b,tstat,pvals,bse,adjR2,yhat = olsnw(Y=Ri,X=pcaf[:,:NF[j]])
        adj = (Ttemp-1)/(Ttemp-1-NF[j])
        temp1 = np.sum((Ri-yhat)**2)*adj
        temp2 = np.sum(Ri**2);
        temp3 = np.sum((Ri-yhat+b[0])**2)*adj/Ttemp
        res_pca[j,i,:] = [temp1, temp2, temp3]
        
        b,tstat,pvals,bse,adjR2,yhat = olsnw(Y=Ri,X=plsf[:,:NF[j]])
        adj = (Ttemp-1)/(Ttemp-1-NF[j])
        temp1 = np.sum((Ri-yhat)**2)*adj
        temp2 = np.sum(Ri**2);
        temp3 = np.sum((Ri-yhat+b[0])**2)*adj/Ttemp
        res_pls[j,i,:] = [temp1, temp2, temp3]

TRf = np.zeros((2,len(NF)))
TRrra = np.zeros((2,len(NF)))
TRpca = np.zeros((2,len(NF)))
TRpls = np.zeros((2,len(NF)))
for j in range(len(NF)):
    TRf[:,j] = [1-res_f[j,:,0].sum()/res_f[j,:,1].sum(), np.sqrt(res_f[j,:,2]).mean()]
    TRrra[:,j] = [1-res_rra[j,:,0].sum()/res_rra[j,:,1].sum(), np.sqrt(res_rra[j,:,2]).mean()]
    TRpca[:,j] = [1-res_pca[j,:,0].sum()/res_pca[j,:,1].sum(), np.sqrt(res_pca[j,:,2]).mean()]
    TRpls[:,j] = [1-res_pls[j,:,0].sum()/res_pls[j,:,1].sum(), np.sqrt(res_pls[j,:,2]).mean()]

# Write out

In [20]:
ls1 = ['TotalR2(%)' for i in range(5)]+['RMSPE (%)' for i in range(5)]
ls2 = [str(i)+' factors' for i in [1,3,5,6,10]]
ls = [ls1, ls2+ls2]
tuples = list(zip(*ls))
col = pd.MultiIndex.from_tuples(tuples, names=['Performance','Model'])
index = ['FF','PCA','PLS','RRA']

In [44]:
### 48 Industry portfolios ###
# Run this section if 48-Industry-portfolios section is chosen in Factor Extraction part

TotalR2_0 = np.array((TRf[0,:],TRpca[0,:],TRpls[0,:],TRrra[0,:]))
PE_0 = np.array((TRf[1,:],TRpca[1,:],TRpls[1,:],TRrra[1,:]))
data = np.concatenate((TotalR2_0,PE_0),axis=1)

res0 = pd.DataFrame(data*100,index=index,columns=col) # target assets: 48 industry portfolios
res0.to_excel('Panel A: 48 Industry Portfolios.xls')

In [45]:
res0

Performance TotalR2(%)                                             RMSPE (%)  \
Model        1 factors  3 factors  5 factors  6 factors 10 factors 1 factors   
FF           51.393095  55.569281  57.765512  58.340052  59.622064  4.459831   
PCA          13.996545  24.589281  29.109066  29.336403  57.342061  6.089327   
PLS          29.535977  55.514035  58.637457  59.382431  61.220821  5.497749   
RRA          53.892899  58.790617  60.852092  61.121727  62.051293  4.341182   

Performance                                           
Model       3 factors 5 factors 6 factors 10 factors  
FF           4.244148  4.122504  4.091064   4.027708  
PCA          5.706049  5.541712  5.532191   4.186298  
PLS          4.274815  4.092748  4.063146   3.964251  
RRA          4.076555  3.974426  3.957533   3.912889

In [40]:
### 202 portfolios ###
# Run this section if 202-portfolios section is chosen in Factor Extraction part
TotalR2_1 = np.array((TRf[0,:],TRpca[0,:],TRpls[0,:],TRrra[0,:]))
PE_1 = np.array((TRf[1,:],TRpca[1,:],TRpls[1,:],TRrra[1,:]))
data = np.concatenate((TotalR2_1,PE_1),axis=1)

res1 = pd.DataFrame(data*100,index=index,columns=col) # target assets: 202 portfolios
res1.to_excel('Panel B: 202 Portfolios.xls')

In [41]:
res1

Performance TotalR2(%)                                             RMSPE (%)  \
Model        1 factors  3 factors  5 factors  6 factors 10 factors 1 factors   
FF           73.310058  85.597389  86.944622  88.301119  88.693842  2.976046   
PCA          24.683966  47.646764  51.412519  51.535811  82.767176  5.022629   
PLS          50.947213  82.319936  86.225735  87.590342  88.693083  4.097931   
RRA          79.716249  86.551418  88.424047  88.762100  89.254748  2.646090   

Performance                                           
Model       3 factors 5 factors 6 factors 10 factors  
FF           2.238080  2.142756  2.056964   2.029469  
PCA          4.237705  4.087485  4.083063   2.483756  
PLS          2.495254  2.226636  2.123262   2.038780  
RRA          2.172331  2.046593  2.025334   1.991098